In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
import time
import copy
import itertools
from scipy.stats import binom
from scipy.optimize import basinhopping

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [2]:
def adjust_angle(depths, n_samples, measurements, theta_found):

    p_o2 = np.cos((2 * depths + 1) * (theta_found / 2.0)) ** 2
    p_o4 = np.cos((2 * depths + 1) * (theta_found / 4.0)) ** 2
    p_same = np.cos((2 * depths + 1) * (theta_found)) ** 2
    p_s2 = np.cos((2 * depths + 1) * (np.pi / 2 - theta_found)) ** 2
    p_s4 = np.cos((2 * depths + 1) * (np.pi / 4 - theta_found)) ** 2
    p_s2_o2 = np.cos((2 * depths + 1) * (np.pi / 2 - theta_found / 2)) ** 2
    p_s4_o2 = np.cos((2 * depths + 1) * (np.pi / 4 - theta_found/ 2)) ** 2

    l_o2 = np.sum(
        np.log(
            [1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_o2[kk]) for
             kk in
             range(len(n_samples))]))
    l_o4 = np.sum(
        np.log(
            [1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_o4[kk]) for
             kk in
             range(len(n_samples))]))
    l_same = np.sum(
        np.log(
            [1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_same[kk]) for
             kk in
             range(len(n_samples))]))
    l_s2 = np.sum(
        np.log(
            [1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_s2[kk]) for
             kk in
             range(len(n_samples))]))
    l_s4 = np.sum(
        np.log(
            [1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_s4[kk]) for
             kk in
             range(len(n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_s2_o2[kk])
                for kk in
                range(len(n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(n_samples[kk] * measurements[kk], n_samples[kk], p_s4_o2[kk])
                for kk in
                range(len(n_samples))]))


    # which_correction = np.argmin([obj_same, obj_s2, obj_s4, obj_o2, obj_s2_o2, obj_s4_o2])
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_found = np.pi / 2.0 - theta_found
    elif which_correction == 2:
        theta_found = np.pi / 4.0 - theta_found
    elif which_correction == 3:
        theta_found =  0.5 * theta_found
    elif which_correction == 4:
        theta_found =  0.25 * theta_found
    elif which_correction == 5:
        theta_found = np.pi / 2.0 - 0.5 * theta_found
    elif which_correction == 6:
        theta_found = np.pi / 4.0 - 0.5 * theta_found

    return np.abs(theta_found)

In [3]:
def objective_function(lp, cos_top, sin_top, ula_signal, esprit):
    sr = lp[:len(lp)//2]
    si = lp[len(lp)//2:]

    signal = ula_signal.signal

    for i,j in enumerate(cos_top):
        signal[j] = sr[i]*np.abs(np.real(signal[j])) + np.imag(signal[j])

    for i,j in enumerate(sin_top):
        signal[j] = np.real(signal[j]) + si[i]*np.abs(np.imag(signal[j]))
    
    # signal = sr * abs_cos + 1.0j * si * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

# Example Implementation

Here we provide a minimal working example demonstrating how to use the code to estimate the amplitude using the ESPIRIT algorithm.

In [4]:
# For reproducibility
seed = 5
np.random.seed(seed)
# Set the per oracle noise parameter (See Eq. 18)
# eta=1e-4
eta=0
# Set the array parameters (See Thm. II.2 and Eq. 12) 
narray = [3, 3, 3, 3, 2, 2, 2, 2]
narray = [2]*10
# Set the actual amplitude
a=0.1
theta = np.arcsin(a)
niter=20

# This sets up the simulation that simulates the measured amplitudes at the various physical locations.
# It uses a C=1.5 value, which corresponds to the sampling schedule given in Eq. 16. The variable C here 
# is the parameter K in the paper.
ula_signal = TwoqULASignal(M=narray, C=1.5)
# Number of Monte Carlo trials used to estimate statistics. We tend to use 500 in the paper. Choose 100 here for speed.
num_mc = 1
thetas = np.zeros(num_mc, dtype = float)
errors = np.zeros(num_mc, dtype = float)

# Sets up the ESPIRIT object to estimate the amplitude
espirit = ESPIRIT()

optimize = True
n_opt = 2

for k in range(num_mc):
    # n_samples = [500]*len(n_samples)
    signal = ula_signal.estimate_signal(ula_signal.n_samples, theta, eta=eta, seed=seed+k)
    # This estimates the covariance matrix of Eq. 8 using the approch given in DOI:10.1109/LSP.2015.2409153
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    # This estimates the angle using the ESPIRIT algorithm
    theta_est, _ = espirit.estimate_theta_toeplitz(R)
    objective = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
    

    if optimize:
        sin_top = np.argsort(-ula_signal.signs_stderr['sin_est'])[:n_opt]
        cos_top = np.argsort(-ula_signal.signs_stderr['cos_est'])[:n_opt]
    
    
        print(ula_signal.signs_stderr['sin_est'][sin_top])
        print(ula_signal.signs_stderr['cos_est'][cos_top])
    
        all_signs = [s for s in itertools.product([1.0, -1.0], repeat=2*n_opt)]
        print(f'objective: {objective}')
    
        for signs in all_signs:
            new_signal = np.array([x for x in ula_signal.signal])
            new_cos_signs = np.array([x for x in ula_signal.signs['cos_est']])
            new_sin_signs = np.array([x for x in ula_signal.signs['sin_est']])
            for j in range(n_opt):
                new_signal[j] = signs[2*j]*np.abs(np.real(new_signal[j])) + signs[2*j+1]*np.abs(np.imag(new_signal[j]))   
                new_cos_signs[cos_top[j]] = signs[2*j]
                new_sin_signs[cos_top[j]] = signs[2*j+1]
                R = ula_signal.get_cov_matrix_toeplitz(new_signal)
                # This estimates the angle using the ESPIRIT algorithm
                theta_new, _ = espirit.estimate_theta_toeplitz(R)
                objective_new = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
    
                if objective_new > objective:
                    cos_signs_found = np.array([x for x in new_cos_signs])
                    sin_signs_found = np.array([x for x in new_sin_signs])
                    theta_est = theta_new
                    objective = objective_new
                    print(f'objective: {objective}')
    
        print(f'sin est:   {ula_signal.signs["sin_est"]}')
        print(f'sin new:   {sin_signs_found}')
        print(f'sin exact: {ula_signal.signs["sin_exact"]}')
        print(f'sin err:   {ula_signal.signs_stderr["sin_est"]}')
        print()
        print(f'cos est:   {ula_signal.signs["cos_est"]}')
        print(f'cos new:   {cos_signs_found}')
        print(f'cos exact: {ula_signal.signs["cos_exact"]}')
        print(f'cos err:   {ula_signal.signs_stderr["cos_est"]}')
        print()
    
    

    # if optimize:
    #     print(f'objective: {objective}')
    #     abs_cos = np.abs(np.real(signal))
    #     abs_sin = np.abs(np.imag(signal))
    #     init_signs_cos = [s for s in ula_signal.signs['cos_est'][cos_top]]
    #     init_signs_sin = [s for s in ula_signal.signs['sin_est'][sin_top]]
    #     init_signs = np.array(init_signs_cos+init_signs_sin)
    #     bounds = tuple((-1.0, 1.0) for _ in range(len(init_signs)))    
    #     res = basinhopping(objective_function, init_signs, niter=niter, T=10.0, stepsize=1.0, 
    #                        minimizer_kwargs={'args': (cos_top, sin_top, ula_signal, espirit), 
    #                                          'bounds': bounds}, disp=True)
    #     x = np.sign(res.x)
    #     # signal = x[:len(init_signs)//2]*abs_cos + 1.0j * x[len(init_signs)//2:] * abs_sin
    #     ula_signal.signs['cos_est'][cos_top] = x[:len(init_signs)//2]
    #     ula_signal.signs['sin_est'][sin_top] = x[len(init_signs)//2:]
    #     signal = np.array(ula_signal.signs['cos_est'])*abs_cos + 1.0j * np.array(ula_signal.signs['sin_est'])*abs_sin
    #     R = ula_signal.get_cov_matrix_toeplitz(signal)
    #     theta_est, _ = espirit.estimate_theta_toeplitz(R)
    #     objective_basin = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
    #     print(f'basin objective: {objective_basin}')
        
    #     signal = np.array(ula_signal.signs['cos_exact'])*abs_cos + 1.0j * np.array(ula_signal.signs['sin_exact'])*abs_sin
    #     R = ula_signal.get_cov_matrix_toeplitz(signal)
    #     _, _ = espirit.estimate_theta_toeplitz(R)
    #     objective_global = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
        
    #     print(f'global objective: {objective_global}')
       
    # for kk in range(len(signal)-1):
    #     signal_new = np.array([x for x in signal])
    #     signal_new[kk+1] = np.conj(signal_new[kk+1])
    #     R = ula_signal.get_cov_matrix_toeplitz(signal_new)
    #     # This estimates the angle using the ESPIRIT algorithm
    #     theta_new, eigs = espirit.estimate_theta_toeplitz(R)
    #     objective_new = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
    #     if objective_new > objective:
    #         objective = objective_new
    #         # print(f'objective new: {objective_new}')
    #         ula_signal.signs['sin_est'][kk]=-1
    #         theta_est = theta_new
        
        
    # R = ula_signal.get_cov_matrix_toeplitz(signal)
    # # This estimates the angle using the ESPIRIT algorithm
    # theta_est, eigs = espirit.estimate_theta_toeplitz(R)
    # objective = np.abs(espirit.eigs[0]) - np.abs(espirit.eigs[1])
    
    theta_est = adjust_angle(ula_signal.depths, ula_signal.n_samples, ula_signal.measurements, theta_est)
    # Estimate the error between estimated a and actual a
    error = np.abs(np.sin(theta)-np.sin(theta_est)) 
    thetas[k] = theta_est            
    errors[k] = error

# Compute the total number of queries. The additional count of n_samples[0] is to 
# account for the fact that the Grover oracle has two invocations of the unitary U, but is 
# preceded by a single invocation of U (see Eq. 2 in paper). This accounts for the shots required
# for that single U operator, which costs half as much as the Grover oracle.
num_queries = 2*np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
# Compute the maximum single query
max_single_query = np.max(ula_signal.depths)

print(ula_signal.signs['cos_est'])
print(ula_signal.signs['cos_exact'])
print()
print(ula_signal.signs['sin_est'])
print(ula_signal.signs['sin_exact'])
print()

print(f'Array parameters: {narray}')
print(f'Depths: {ula_signal.depths}')
print(f'Samples: {ula_signal.n_samples}')
print(f'Number of queries: {num_queries}')
print(f'theta: {theta/np.pi}')
print(f'Ave theta estimated: {np.mean(thetas)/np.pi}')
print(f'a = {a}; a_est = {np.sin(np.mean(thetas))}')
print(f'Max Single Query: {max_single_query}')
print(f'99% percentile: {np.percentile(errors, 99):e}')
print(f'95% percentile: {np.percentile(errors, 95):e}')
print(f'68% percentile: {np.percentile(errors, 68):e}')
print(f'99% percentile constant: {np.percentile(errors, 99)*num_queries:f}')
print(f'95% percentile constant: {np.percentile(errors, 95)*num_queries:f}')
print(f'68% percentile constant: {np.percentile(errors, 68)*num_queries:f}')
print(f'99% percentile max constant: {np.percentile(errors, 99)*max_single_query:f}')
print(f'95% percentile max constant: {np.percentile(errors, 95)*max_single_query:f}')
print(f'68% percentile max constant: {np.percentile(errors, 68)*max_single_query:f}')
print()


[0.125 0.08 ]
[0.22222222 0.125     ]
objective: 259.06790295091054
objective: 259.06790295091065
objective: 259.0679029509108
objective: 369.28551098484695
objective: 369.285510984848
objective: 369.2855109848483
objective: 369.28551098484854
sin est:   [ 1 -1 -1 -1  1  1  1 -1  1 -1  1]
sin new:   [ 1 -1 -1 -1  1  1  1 -1  1 -1  1]
sin exact: [ 1  1  1  1  1 -1  1  1  1  1  1]
sin err:   [0.         0.00414815 0.01494169 0.03414352 0.04658152 0.
 0.01367188 0.02314815 0.08       0.07407407 0.125     ]

cos est:   [ 1  1  1  1  1 -1  1  1  1  1 -1]
cos new:   [ 1  1  1  1  1 -1  1  1  1 -1 -1]
cos exact: [ 1  1  1  1 -1 -1  1  1  1  1 -1]
cos err:   [0.         0.00414815 0.01202624 0.03414352 0.040571   0.
 0.01367188 0.02314815 0.048      0.22222222 0.125     ]

[ 1  1  1  1  1 -1  1  1  1  1 -1]
[ 1  1  1  1 -1 -1  1  1  1  1 -1]

[ 1 -1 -1 -1  1  1  1 -1  1 -1  1]
[ 1  1  1  1  1 -1  1  1  1  1  1]

Array parameters: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Depths: [  0   1   2   4   8  16 